<a href="https://colab.research.google.com/github/yviler/CompvisAssignmentII/blob/main/compvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Moehammad Azzriel Ilham

21/477994/PA/20724

CSB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Dataset/COVID-19/training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Dataset/COVID-19/testing',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 150, 150, 3], [None])
).repeat()

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 150, 150, 3], [None])
).repeat()


Found 109 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


This code is for preparing the data to be used for training and testing a model that detects COVID-19 from chest X-ray images. It rescales the pixel values of the images to be between 0 and 1, and applies several image augmentation techniques to increase the number of training examples and make the model more robust.

The ImageDataGenerator class is used to generate batches of training and testing data. The generator loads images from a specified directory, applies the specified image augmentation techniques, and creates batches of images with their corresponding labels (COVID-19 positive or negative).

The tf.data.Dataset class is then used to convert the data from the generator into a format that can be used by the model during training. The lambda function is used to create a generator that generates batches of data indefinitely, and the repeat() function is used to ensure that the generator continues to generate data indefinitely during training.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import RMSprop

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


This defines teh model for image classification using the Keras library. The model consists of several layers of convolutional and pooling operations, followed by a flatten layer, two dense layers, and a final output layer. The model uses the RMSprop optimizer with a learning rate of 0.0001 and the binary crossentropy loss function. It also measures the accuracy metric during training.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Dataset/COVID-19/training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Dataset/COVID-19/testing',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

history = model.fit(
    train_dataset,
    steps_per_epoch=100,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=2)

model.save('covid_classifier_model.h5')


Found 109 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 400s 4s/step - loss: 0.5712 - accuracy: 0.7248 - val_loss: 0.6995 - val_accuracy: 0.5745
Epoch 2/10
100/100 [==============================] - 379s 4s/step - loss: 0.5139 - accuracy: 0.7622 - val_loss: 0.6761 - val_accuracy: 0.6809
Epoch 3/10
100/100 [==============================] - 392s 4s/step - loss: 0.4489 - accuracy: 0.7960 - val_loss: 0.6728 - val_accuracy: 0.6809
Epoch 4/10
100/100 [==============================] - 383s 4s/step - loss: 0.4020 - accuracy: 0.8169 - val_loss: 0.6499 - val_accuracy: 0.6809
Epoch 5/10
100/100 [==============================] - 379s 4s/step - loss: 0.3427 - accuracy: 0.8492 - val_loss: 0.8044 - val_accuracy: 0.5957
Epoch 6/10
100/100 [==============================] - 381s 4s/step - loss: 0.3193 - accuracy: 0.8569 - val_loss: 0.7411 - val_accuracy: 0.5745
Epoch 7/10
100/100 [==============================] - 387s 4s

Trains the CNN to the dataset we have established

The first section imports the necessary modules, defines the architecture of the neural network model, and compiles it.

The second section uses the ImageDataGenerator class to generate augmented images for the training and testing datasets. The train_datagen rescales the pixel values of the images and applies data augmentation techniques such as shearing, zooming, and horizontal flipping to increase the number of training samples. The test_datagen only rescales the pixel values of the images.

The third section uses the flow_from_directory method to load the training and testing datasets from the specified directories, resizes the images to 150x150 pixels, and generates batches of 32 images for each batch.

The fourth section trains the model on the training dataset, specifying the number of steps per epoch, the number of epochs, the validation dataset, and the number of validation steps. After training, the model is saved as a h5 file.

# The Output 
The output shows the same metrics for the remaining epochs. Overall, the model have achieved an accuracy of about 94% on the training set, and about 66% on the validation set. However, there is a large gap between the training and validation accuracy, which indicates that the model may be overfitting to the training data.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model('covid_classifier_model.h5')

img_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/COVID-19/walao/kjr-21-e24-g001-l-a.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)
if prediction[0] < 0.5:
    print("Non-COVID")
else:
    print("COVID")


1/1 [==============================] - 0s 209ms/step
Non-COVID


loads the trained model for detecting COVID-19 from an image, using Keras. It then loads an image from a file and prepares it for prediction by resizing it and converting it to a numpy array. Finally, it uses the loaded model to make a prediction on the image and prints whether the image is predicted to show signs of COVID-19 or not.

# The Output
This particular output shows that the image in the path is non-covid which indeed is non-covid